In [3]:
import tweepy
import plotly
import datetime
import pytz
import re

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from nltk.tokenize  import word_tokenize
from nltk.corpus import stopwords

from string import punctuation

# Keys
access_token = '1258532548134797313-w24ae6Fc7uIHO0jXk40akPLrFfUvMW'

access_token_secret = 'L8PwULD3RHMYHnxc7cJlvZSYtXTQORc0ElEFIfL1S7V21'

consumer_key = 'YuJdN38YPbdwFJpAUooTHhGbh'

consumer_key_secret = 'IOzsqk0Vly9Y90xhXqXQqQ1tsyLS11zgVTjGnpdMTkYJzKH2Dl'

# Start the API 
auth = tweepy.OAuthHandler(consumer_key,consumer_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [4]:
tweets_list = []

for t in tweepy.Cursor(api.user_timeline, id = 'GGallegos24').items():
    tweet_id = t.id
    try:
        # Create dictionary for specific follower and save data in there
        tweet = {}
        tweet['id'] = t.id
        tweet['created_at'] = t.created_at 
        tweet['tweet'] = t.text   

    except:
        pass
    
    tweets_list.append(tweet)


In [5]:
df = pd.DataFrame(tweets_list)
print(df.head(5))


id          created_at  \
0  1275423803519229952 2020-06-23 13:41:45   
1  1275422386343600131 2020-06-23 13:36:07   
2  1275421552927014916 2020-06-23 13:32:49   
3  1275421111237435393 2020-06-23 13:31:03   
4  1275420558793064448 2020-06-23 13:28:52   

                                               tweet  
0  La carta del Presidente @nayibbukele la interp...  
1  Estaba viendo el último tweet que puso Eduardo...  
2  Tenemos un Presidente @nayibbukele @Presidenci...  
3  Se los he dicho a este grupito que anda con @J...  
4  Sobre el Proceso de Destitución que andan prom...  


In [6]:
# Define functions for cleaning data
stop_words = stopwords.words('spanish')

# Clean text in tweets
def remove_content(text):
    text = text.lower() #bring all letters to lowercase
    
    text  = text.replace('rt', '') #remove rt
    text = re.sub(r"http\S+", "", text) #remove urls
    text = re.sub(r'\S+\.com\S+','',text) #remove urls
    text = re.sub(r'\@\w+','',text) #remove mentions
    text = re.sub(r'\#\w+','',text) #remove hashtags
    text = re.sub(r'[^\w\s]','',text) #remove punctuation marks

    return text

# Remove stopwords
def remove_stopwords(text):
    # Clean text before removing stopwords
    text = remove_content(text)

    word_tokens = word_tokenize(text)

    filtered_tweet = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w)

    return " ".join(filtered_tweet)

df['cleaned_tweet'] = df['tweet'].apply(remove_stopwords)
print(df.head())

id          created_at  \
0  1275423803519229952 2020-06-23 13:41:45   
1  1275422386343600131 2020-06-23 13:36:07   
2  1275421552927014916 2020-06-23 13:32:49   
3  1275421111237435393 2020-06-23 13:31:03   
4  1275420558793064448 2020-06-23 13:28:52   

                                               tweet  \
0  La carta del Presidente @nayibbukele la interp...   
1  Estaba viendo el último tweet que puso Eduardo...   
2  Tenemos un Presidente @nayibbukele @Presidenci...   
3  Se los he dicho a este grupito que anda con @J...   
4  Sobre el Proceso de Destitución que andan prom...   

                                       cleaned_tweet  
0  caa presidente interpreto pedido auxilio facul...  
1  viendo último tweet puso eduardo navas 13 juni...  
2  presidente quiere hacer cosas diferentes puebl...  
3  dicho grupito anda cualquier ocurra querer des...  
4  proceso destitución andan promoviendo luz alea...  


In [34]:
# Define functions for parsing created_at

## Change timezone in created_at
def change_tz(created_at):
    utc_moment = created_at.replace(tzinfo = pytz.utc)
    return(utc_moment.astimezone(pytz.timezone('America/El_Salvador')))

df['created_at'] = df['created_at'].apply(change_tz)

## Parse dates
def parse_created_at(created_at):
    return(datetime.datetime.strftime(created_at,'%Y-%m-%d'))
def parse_weekday(created_at):
    return(created_at.strftime('%A'))
def parse_date(created_at):
    return(created_at.day)
def parse_month(created_at):
    return(created_at.strftime('%B'))
def parse_year(created_at):
    return(created_at.year)

## Parse hours
def parse_hour(created_at):
    return(created_at.hour)

df['hour'] = df['created_at'].apply(parse_hour)
df['date'] = df['created_at'].apply(parse_date)
df['weekday'] = df['created_at'].apply(parse_weekday)
df['month'] = df['created_at'].apply(parse_month)
df['year'] = df['created_at'].apply(parse_year)
df['created_at_date'] = df['created_at'].apply(parse_created_at)

## Add column to sort weekdays by numbers
def sort_weekday(created_at):
    n= created_at.weekday()
    return((-1*n) + 7)

df['sort_weekday'] = df['created_at'].apply(sort_weekday)

## Add column to sort months
def sort_month(created_at):
    return(created_at.month)

df['sort_month'] = df['created_at'].apply(sort_month)

# Tweet length
df['tweet_length'] = df['tweet'].apply(len)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

def plot_topn(sentences,ngram_range = (1,3), top = 20, firstword =''):
    c = CountVectorizer(ngram_range=ngram_range)
    X = c.fit_transform(sentences)
    words=pd.DataFrame(X.sum(axis=0),columns=c.get_feature_names()).T.sort_values(0,ascending=False).reset_index()
    res=words[words['index'].apply(lambda x: firstword in x)].head(top)
    pl=px.bar(res, x=0,y='index', orientation ='h')
    pl.update_layout(yaxis_title = 'Frases', xaxis_title = 'Frencuencia',title = (str(top) + ' Frases Mas Comunes con ' + str(ngram_range[1]) + ' Palabras'))
    pl['layout']['yaxis']['autorange'] = 'reversed'
    pl.show()

tweet_list = df['cleaned_tweet'].tolist()
plot_topn(tweet_list,(3,3))

In [9]:
from classifier import *

cfl = SentimentClassifier()

def sentiment_analysis(text):
    return(cfl.predict(text))

df['sentiment'] = df['tweet'].apply(sentiment_analysis)

In [33]:
df_2 = df.groupby(['weekday','hour','sort_weekday','sentiment']).mean().reset_index()

df_2 = df_2.sort_values(by = 'sort_weekday')
fig = go.Figure(data=go.Heatmap(
        x = df_2['hour'],
        y = df_2['weekday'],
        z = df_2['sentiment']
        ))

fig.update_layout(title = 'Sentimiento Promedio de Tuit por Hora y Dia', xaxis_title='Hora', yaxis_title='Dia de la Semana')

In [37]:
fig = px.scatter(
    df,
    x = 'tweet_length',
    y = 'sentiment',
)

fig.show()

In [44]:
from vaderSentiment import *

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

text = 'You are smart'

analyzer = SentimentIntensityAnalyzer()

vs = analyzer.polarity_scores(text)

print(str(vs))


{'neg': 0.0, 'neu': 0.426, 'pos': 0.574, 'compound': 0.4019}


In [61]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
spanish_stopwords = stopwords.words('spanish')


[nltk_data] Downloading package stopwords to C:\Users\Abraham Mena
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
from string import punctuation
non_words = list(punctuation)

# Add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))


In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

stemmer = SnowballStemmer('spanish')

def stem_tokens(tokens,stemmer):
    stemmed = []

    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    #remove non letters
    text = ''.join([c for c in text if c not in non_words])
    #tokenize
    tokens = word_tokenize(text)

    #stem
    try:
        stems = stem_tokens(tokens,stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

In [158]:
# Open up database to train 
pd.set_option('max_colwidth',1000)
from lxml import objectify

## Open General Tweets Train Tagged
try:
    general_tweets_corpus_train = pd.read_csv(r'C:\Users\Abraham Mena V\OneDrive\Professional Development\Proyectos\Programacion\Python\Twitter\NB_tuits\training_databases\general-tweets-train-tagged.xml',encoding = 'utf-8')
except:
    xml  = objectify.parse(open(r'C:\Users\Abraham Mena V\OneDrive\Professional Development\Proyectos\Programacion\Python\Twitter\NB_tuits\training_databases\general-tweets-train-tagged.xml', encoding='utf-8'))
    root = xml.getroot()
    general_tweets_corpus_train = pd.DataFrame(columns = ['content','polarity','agreement'])

    data = []
    tweets = root.getchildren()
    
    for i in range(len(tweets)):
        tweet = tweets[i]

        t = {}
        t['content'] = tweet.content.text
        t['polarity'] = tweet.sentiments.polarity.value.text
        t['agreement'] = tweet.sentiments.polarity.type.text
        
        data.append(t)
    
tweets_corpus = pd.DataFrame(data)

tweets_corpus.sample(10)


,content,polarity,agreement
3495,Aaaaaaaaaaaaaaaaah...! Por dios ya lo he sacado! Que nervios... JAJAJAJAJA,P,AGREEMENT
3916,A los que defienden a Garzon y pretenden saltarase la ley. editorial de El País de 1995: http://t.co/HX340iYg muy recomendable lectura,NEU,DISAGREEMENT
6134,"@javierarenas_pp #PROPONE: Reformar la Ley de Publicidad de la Junta para que sea un instrumento de información, y no de propaganda.",P,AGREEMENT
6908,@rrodriguezmaeso Empanado o rebozado?,N,AGREEMENT
3498,Cataluña y Valencia al frente... T @pmartind: 'The Economist' alerta sobre el déficit de las CCAA http://t.co/FFpbuqiT vía @el_pais,N,AGREEMENT
6176,"Romney gana ampliamente en Illinois. ""soy el único capaz derrotar a Obama""",P,AGREEMENT
5796,¡Noticias descombacantes! está disponible! http://t.co/GtjLld7K,P,AGREEMENT
3536,"Feflexión sobre Rafa Nada: ¿Era legal lo q hacía para pagar menos impuestos? Si. Luego el escándalo no es Rafa, sino q eso se pudiera hacer.",N,DISAGREEMENT
1937,"“@martinez_xabier: en su epoka ya abia moviles¿?” Claaaro, creo que había hasta diccionarios.",NONE,AGREEMENT
3291,RT @PwC_Spain: Estado ahorraría más d 1.000 mm. al año con la puesta en marcha de centros de servicios compartidos http://t.co/0UmOBSyT #PwC,P,AGREEMENT


In [160]:
# Filtrar tweets en los que no hay concordancia en agreement y no son polarizadores
tweets_corpus = tweets_corpus.query('agreement != "DISAGREEMENT" and polarity != "NONE"')

#Remover links
tweets_corpus = tweets_corpus[-tweets_corpus.content.str.contains('^http.*$')]

tweets_corpus.shape

(4995, 3)

In [161]:
# Descargard stopwrods en espa;ol
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to C:\Users\Abraham Mena
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [163]:
from string import punctuation
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

stemmer = SnowballStemmer('spanish')

def stem_tokens(tokens,stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
    
def tokenize(text):
    # remove non letters
    text = ''.join([c for c in text if c not in non_words])
    # tokenize
    tokens =  word_tokenize(text)

    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems